# Data Pre-Processing
## Import Libraries

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Import Cleaned Dataset

In [7]:
df = pd.read_csv('Data/cleaned.csv')

In [8]:
df.head()

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,inn00001,2,0,1,2,meal plan 1,0,room_type 1,224,2017,10,2,offline,0,0,0,65.00,0,not_canceled
1,inn00002,2,0,2,3,not selected,0,room_type 1,5,2018,11,6,online,0,0,0,106.68,1,not_canceled
2,inn00003,1,0,2,1,meal plan 1,0,room_type 1,1,2018,2,28,online,0,0,0,60.00,0,canceled
3,inn00004,2,0,0,2,meal plan 1,0,room_type 1,211,2018,5,20,online,0,0,0,100.00,0,canceled
4,inn00005,2,0,1,1,not selected,0,room_type 1,48,2018,4,11,online,0,0,0,94.50,0,canceled


In [9]:
df.columns

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status'],
      dtype='object')

# Data Preprocessing

## Train-Test Split

There are two possible methods for train-test split:

1. **Random Split (80/20 Rule)**: This method involves randomly dividing the dataset into training and test sets, typically with a split ratio of 80% for training and 20% for testing.
2. **Date-Based Split**: This method splits the data based on the booking date, where entries from earlier dates are assigned to the training set and those from later dates are assigned to the test set. 

We will adopt method 2 because it closely mimics real-life scenarios, where predictions are made using historical data to forecast future outcomes. Additionally, the size of the training and test sets will be approximately 80:20, ensuring a balanced distribution.

In [10]:
# create booking_date
temp = df.rename(columns={
    'arrival_year': 'year',
    'arrival_month': 'month',
    'arrival_date': 'day'
})

df['booking_date'] = pd.to_datetime(temp[['year', 'month', 'day']], errors='coerce')
df['booking_date'] = df['booking_date'] - pd.Series(
    [pd.Timedelta(i, 'd') for i in df.lead_time],
    index=df.index
)

In [11]:
max(df['booking_date'])

Timestamp('2018-12-30 00:00:00')

In [12]:
# split based on booking_date
train = df[df['booking_date']<'2018-07-01'].reset_index(drop=True)
test = df[df['booking_date']>='2018-07-01'].reset_index(drop=True)

In [13]:
print(f'Size of training set: {len(train)}')
print(f'Size of test set: {len(test)}')

Size of training set: 28062
Size of test set: 8176


## Preprocessing on Categorical Variables

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36238 entries, 0 to 36237
Data columns (total 20 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Booking_ID                            36238 non-null  object        
 1   no_of_adults                          36238 non-null  int64         
 2   no_of_children                        36238 non-null  int64         
 3   no_of_weekend_nights                  36238 non-null  int64         
 4   no_of_week_nights                     36238 non-null  int64         
 5   type_of_meal_plan                     36238 non-null  object        
 6   required_car_parking_space            36238 non-null  int64         
 7   room_type_reserved                    36238 non-null  object        
 8   lead_time                             36238 non-null  int64         
 9   arrival_year                          36238 non-null  int64         
 10

From the above info, we can see that in the dataset, we have many categorical attributes. In order to perform classification, additional data preprocessing has to be done. Let's take a look at the attributes that are non-numerical (Categorical)

In [15]:
df.select_dtypes(exclude=['int64', 'float64', 'int32', 'float32']).head()

,Booking_ID,type_of_meal_plan,room_type_reserved,market_segment_type,booking_status,booking_date
0,inn00001,meal plan 1,room_type 1,offline,not_canceled,2017-02-20
1,inn00002,not selected,room_type 1,online,not_canceled,2018-11-01
2,inn00003,meal plan 1,room_type 1,online,canceled,2018-02-27
3,inn00004,meal plan 1,room_type 1,online,canceled,2017-10-21
4,inn00005,not selected,room_type 1,online,canceled,2018-02-22


Let's also explore what are the possible values given the different categorical attributes. From this, we can identify nominal and ordinal categories, this is important since we would use different preprocessing method depending on the category type.

For example, for <b>ordinal data</b> (e.g. Low, med high), there is a natural order to the data, and hence we could replace the values using ordinal encoding, where we map each category with numbers to represent that order.

For <b>nominal data</b> (e.g. Red Blue Green), there is no natural order, we cannot simply apply the same strategy as above, since that would imply an artificial order for the data, which is not true. In this case, a strategy we could use is one-hot encoding.

In [16]:
print("Booking_ID unique values: ", df['Booking_ID'].unique(), "\n")
print("type_of_meal_plan unique values: ", df['type_of_meal_plan'].unique(), "\n")
print("room_type_reserved unique values: ", df['room_type_reserved'].unique(), "\n")
print("market_segment_type unique values: ", df['market_segment_type'].unique(), "\n")
print("booking_status unique values: ", df['booking_status'].unique(), "\n")

Booking_ID unique values:  ['inn00001' 'inn00002' 'inn00003' ... 'inn36273' 'inn36274' 'inn36275'] 

type_of_meal_plan unique values:  ['meal plan 1' 'not selected' 'meal plan 2' 'meal plan 3'] 

room_type_reserved unique values:  ['room_type 1' 'room_type 4' 'room_type 2' 'room_type 6' 'room_type 5'
 'room_type 7' 'room_type 3'] 

market_segment_type unique values:  ['offline' 'online' 'corporate' 'aviation' 'complementary'] 

booking_status unique values:  ['not_canceled' 'canceled'] 



### Based on results, we can see a few observations:
1. Each Booking_ID is unique for each row, this attribute is only used as an identifier to a row, this also means that these are arbitrary values that do not contain any information that helps the model learn patterns related to the target variable, hence we can actually exclude this attribute during our model training.
2. All of the categories appears to be of nominal data type, where there is no real natural order, hence a good strategy we can use here would be one-hot encoding. One-hot encoding creates a binary column for each category and gives a 1 or 0 value to indicate true or false.
3. For Booking_Status, this is actually our target attribute, it only has two possible values (canceled and not_canceled), in this case, we can just use a label encoding to transform the values into a binary value (1 and 0).

In [17]:
# Drop the Booking_ID column as this will not be helpful in training the classification model.
train = train.drop('Booking_ID', axis = 1)
test = test.drop('Booking_ID', axis = 1)

In [18]:
# Encode booking_status
train['booking_status'] = train['booking_status'].apply(lambda x: 1 if x=='canceled' else 0)
test['booking_status'] = test['booking_status'].apply(lambda x: 1 if x=='canceled' else 0)

Hence we have the remaining categorical columns to preprocess:

In [19]:
# Categoriy columns that needs to be preprocessed for classification
cat_cols_to_be_encoded = ['type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 'arrival_month']

# Category columns that are already in the binary format (distinct 0 and 1 values)
cat_cols_already_in_the_right_format = ['required_car_parking_space', 'repeated_guest']


There are several ways for encoding categorical varaibles:
1. <b>OrdinalEncoder</b>: Preserves the ordinal relationship between categories, useful when the categorical variables have an inherent order
2. <b>OneHotEncoder</b>: Creates binary columns for each category, indicating the presence or absence of each category, suitable for nominal data with no ordinal relationship
3. <b>TargetEncoder</b>: Replaces categories with the mean of the target variable for each category, useful for high-cardinality categorical variables
4. <b>FrequencyEncoding</b>: Encodes categorical variables based on the frequency of each category in the dataset, useful when the frequency of occurrence is informative, but can lead to information loss

For the above variables, they have small cardinalities and are nominal data, so OneHotEncoding is the most suitable method.

In [20]:
# fit on training data
enc = OneHotEncoder(handle_unknown='error')
enc.fit(train[cat_cols_to_be_encoded])
train[enc.get_feature_names_out()] = enc.transform(train[cat_cols_to_be_encoded]).toarray()

In [21]:
# transform test data
test[enc.get_feature_names_out()] = enc.transform(test[cat_cols_to_be_encoded]).toarray()

In [22]:
# store the categorical column names after encoding
encoded_cat_cols = enc.get_feature_names_out()

# append the encoded_cat_cols with the columns that are already in the right format
encoded_cat_cols = np.append(encoded_cat_cols, cat_cols_already_in_the_right_format)
encoded_cat_cols

array(['type_of_meal_plan_meal plan 1', 'type_of_meal_plan_meal plan 2',
       'type_of_meal_plan_meal plan 3', 'type_of_meal_plan_not selected',
       'room_type_reserved_room_type 1', 'room_type_reserved_room_type 2',
       'room_type_reserved_room_type 3', 'room_type_reserved_room_type 4',
       'room_type_reserved_room_type 5', 'room_type_reserved_room_type 6',
       'room_type_reserved_room_type 7', 'market_segment_type_aviation',
       'market_segment_type_complementary',
       'market_segment_type_corporate', 'market_segment_type_offline',
       'market_segment_type_online', 'arrival_month_1', 'arrival_month_2',
       'arrival_month_3', 'arrival_month_4', 'arrival_month_5',
       'arrival_month_6', 'arrival_month_7', 'arrival_month_8',
       'arrival_month_9', 'arrival_month_10', 'arrival_month_11',
       'arrival_month_12', 'required_car_parking_space', 'repeated_guest'],
      dtype=object)

For now, all categories are kept to ensure interpretability. To avoid multicollinearity issues in linear models, extra steps can be done to remove some of the attributes.

In [23]:
# Quick Peek at the dataset after categorical preprocessing to verify one hot encoding is successful.
train.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,...,arrival_month_3,arrival_month_4,arrival_month_5,arrival_month_6,arrival_month_7,arrival_month_8,arrival_month_9,arrival_month_10,arrival_month_11,arrival_month_12
0,2,0,1,2,meal plan 1,0,room_type 1,224,2017,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0,2,1,meal plan 1,0,room_type 1,1,2018,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0,0,2,meal plan 1,0,room_type 1,211,2018,5,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,0,1,1,not selected,0,room_type 1,48,2018,4,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2,0,0,2,meal plan 2,0,room_type 1,346,2018,9,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Feature Engineering on Numerical Variables

For machine learning algorithm that requires gradient descent or distance measurement, feature scaling is necessary. There are several ways of scaling:
1. **Min-Max Scaling**: Scales the data to a fixed range, usually between 0 and 1, useful when the distribution of data is uniform or when the algorithm requires input features to be on a similar scale
2. **Standardization**: Scales the data to have a mean of 0 and a standard deviation of 1, preserves the shape of the original distribution and is less affected by outliers

We will experiment with both methods and compare the performance in the following sections.

In [24]:
num_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights', 'no_of_week_nights', 'no_of_previous_cancellations', 
            'no_of_previous_bookings_not_canceled', 'no_of_special_requests', 'lead_time', 'avg_price_per_room']

### Minmax Scaling

In [34]:
# fit on training data
minmax = MinMaxScaler()
minmax.fit(train[num_cols])
train[[x+'_minmax' for x in num_cols]] = minmax.transform(train[num_cols])

In [35]:
# transform test data
test[[x+'_minmax' for x in num_cols]] = minmax.transform(test[num_cols])

In [36]:
# store the numerical column names after scaling
minmax_num_cols = [x+'_minmax' for x in num_cols]
minmax_num_cols

['no_of_adults_minmax',
 'no_of_children_minmax',
 'no_of_weekend_nights_minmax',
 'no_of_week_nights_minmax',
 'no_of_previous_cancellations_minmax',
 'no_of_previous_bookings_not_canceled_minmax',
 'no_of_special_requests_minmax',
 'lead_time_minmax',
 'avg_price_per_room_minmax']

### Standardization

In [25]:
# fit on training data
scaler = StandardScaler()
scaler.fit(train[num_cols])
train[[x+'_scaled' for x in num_cols]] = scaler.transform(train[num_cols])

In [27]:
# transform test data
test[[x+'_scaled' for x in num_cols]] = scaler.transform(test[num_cols])

In [28]:
# store the numerical column names after scaling
scaled_num_cols = [x+'_scaled' for x in num_cols]
scaled_num_cols

['no_of_adults_scaled',
 'no_of_children_scaled',
 'no_of_weekend_nights_scaled',
 'no_of_week_nights_scaled',
 'no_of_previous_cancellations_scaled',
 'no_of_previous_bookings_not_canceled_scaled',
 'no_of_special_requests_scaled',
 'lead_time_scaled',
 'avg_price_per_room_scaled']

## Examination with Logistic Regression

Train and test set will first be prepared for model building. Then a logistic regression will be fitted to the 2 sets of processed data and compared the performance:
1. encoded categorical variables + minmax scaled numerical variables
2. encoded categorical variables + standardized numerical variables 

In [47]:
X_train = train.drop(['booking_status'], axis = 1)
Y_train = train['booking_status']
X_test = test.drop(['booking_status'], axis = 1)
Y_test = test['booking_status']

In [48]:
X_train.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,...,avg_price_per_room_minmax,no_of_adults_robust,no_of_children_robust,no_of_weekend_nights_robust,no_of_week_nights_robust,no_of_previous_cancellations_robust,no_of_previous_bookings_not_canceled_robust,no_of_special_requests_robust,lead_time_robust,avg_price_per_room_robust
0,2,0,1,2,meal plan 1,0,room_type 1,224,2017,10,...,0.120370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.145038,-0.844327
1,1,0,2,1,meal plan 1,0,room_type 1,1,2018,2,...,0.111111,-1.0,0.0,0.5,-0.5,0.0,0.0,0.0,-0.557252,-0.976253
2,2,0,0,2,meal plan 1,0,room_type 1,211,2018,5,...,0.185185,0.0,0.0,-0.5,0.0,0.0,0.0,0.0,1.045802,0.079156
3,2,0,1,1,not selected,0,room_type 1,48,2018,4,...,0.175000,0.0,0.0,0.0,-0.5,0.0,0.0,0.0,-0.198473,-0.065963
4,2,0,0,2,meal plan 2,0,room_type 1,346,2018,9,...,0.212963,0.0,0.0,-0.5,0.0,0.0,0.0,1.0,2.076336,0.474934


### encoded categorical variables + scaled numerical variables

In [62]:
logRegModel = LogisticRegression(max_iter=10000).fit(X_train[list(encoded_cat_cols)+list(scaled_num_cols)], Y_train)

# Predict on the training and test data
Y_train_pred = logRegModel.predict(X_train[list(encoded_cat_cols)+list(scaled_num_cols)])
Y_test_pred = logRegModel.predict(X_test[list(encoded_cat_cols)+list(scaled_num_cols)])

In [63]:
# Calculate general metrics for the train set
print("**Training Set Metrics**")
print("Accuracy \t:", accuracy_score(Y_train, Y_train_pred))
print("Precision \t:", precision_score(Y_train, Y_train_pred))
print("Recall \t\t:", recall_score(Y_train, Y_train_pred))
print("F1 Score \t:", f1_score(Y_train, Y_train_pred))

print() # New Line

# Calculate general metrics for the test set
print("**Test Set Metrics**")
print("Accuracy \t:", accuracy_score(Y_test, Y_test_pred))
print("Precision \t:", precision_score(Y_test, Y_test_pred))
print("Recall \t\t:", recall_score(Y_test, Y_test_pred))
print("F1 Score \t:", f1_score(Y_test, Y_test_pred))

**Training Set Metrics**
Accuracy 	: 0.8022949183949826
Precision 	: 0.7495168563452866
Recall 		: 0.6846802667712829
F1 Score 	: 0.7156330087134802

**Test Set Metrics**
Accuracy 	: 0.815068493150685
Precision 	: 0.6009501187648456
Recall 		: 0.30083234244946494
F1 Score 	: 0.4009508716323296


### encoded categorical variables + minmax scaled numerical variables

In [40]:
logRegModel = LogisticRegression(max_iter=10000).fit(X_train[list(encoded_cat_cols)+list(minmax_num_cols)], Y_train)

# Predict on the training and test data
Y_train_pred = logRegModel.predict(X_train[list(encoded_cat_cols)+list(minmax_num_cols)])
Y_test_pred = logRegModel.predict(X_test[list(encoded_cat_cols)+list(minmax_num_cols)])

In [41]:
# Calculate general metrics for the train set
print("**Training Set Metrics**")
print("Accuracy \t:", accuracy_score(Y_train, Y_train_pred))
print("Precision \t:", precision_score(Y_train, Y_train_pred))
print("Recall \t\t:", recall_score(Y_train, Y_train_pred))
print("F1 Score \t:", f1_score(Y_train, Y_train_pred))

print() # New Line

# Calculate general metrics for the test set
print("**Test Set Metrics**")
print("Accuracy \t:", accuracy_score(Y_test, Y_test_pred))
print("Precision \t:", precision_score(Y_test, Y_test_pred))
print("Recall \t\t:", recall_score(Y_test, Y_test_pred))
print("F1 Score \t:", f1_score(Y_test, Y_test_pred))

**Training Set Metrics**
Accuracy 	: 0.8027581783194355
Precision 	: 0.7524645217202903
Recall 		: 0.681247548058062
F1 Score 	: 0.7150872497040202

**Test Set Metrics**
Accuracy 	: 0.8180039138943248
Precision 	: 0.6246786632390745
Recall 		: 0.28894173602853745
F1 Score 	: 0.39512195121951216


The results indicate that standardizing the numerical variables leads to slightly better performance. This improvement could be attributed to the robustness of standardization against extreme values present in our dataset, as well as its independence from the requirement for data to follow a specific distribution.

Having completed the preprocessing steps, the next task is to address the issue of data imbalance.

## Save Processed Datasets

In [86]:
original_cols = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests']

In [89]:
X_train[list(encoded_cat_cols)+list(scaled_num_cols)+original_cols].to_csv('Data/X_train.csv', index = False)
Y_train.to_csv('Data/y_train.csv', index = False)
X_test[list(encoded_cat_cols)+list(scaled_num_cols)+original_cols].to_csv('Data/X_test.csv', index = False)
Y_test.to_csv('Data/y_test.csv', index = False)

In [90]:
# store the column names used for model building
columns = pd.DataFrame({'Columns':list(encoded_cat_cols)+list(scaled_num_cols)})

In [92]:
columns.to_csv('Data/basic_model_columns.csv', index=False)